In [1]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

/home/user/anaconda3/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/user/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


In [2]:
N_ENVS = 16

env = make_vec_env("G1GoalDist-v1", N_ENVS)

/home/user/anaconda3/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/user/anaconda3/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
model = SAC(
    "MlpPolicy", 
    env, 
    verbose=1,
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=2,
    ),
    batch_size=2048,
    learning_rate=1e-4,
    # learning_starts=1000,
    tensorboard_log="./logs/sac/"
)

Using cuda device


In [10]:
def reptile(n_reptile_iterations=100000, n_sac_iterations=50000):
    for _ in range(n_sac_iterations):
        tasks = model.env.env_method('sample_tasks', (N_ENVS), indices=0)[0]
        for i in range(N_ENVS):
            model.env.env_method('reset_task', (tasks[i]), indices=[i])

        model.learn(
            total_timesteps=n_sac_iterations, 
            log_interval=100,
            tb_log_name='reptile',
            reset_num_timesteps=False,
        )

In [11]:
reptile()

Logging to ./logs/sac/reptile_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 279       |
|    ep_rew_mean     | -1.76e+03 |
| time/              |           |
|    episodes        | 100       |
|    fps             | 554       |
|    time_elapsed    | 66        |
|    total_timesteps | 36880     |
| train/             |           |
|    actor_loss      | 29.6      |
|    critic_loss     | 15.1      |
|    ent_coef        | 0.8       |
|    ent_coef_loss   | -3.51     |
|    learning_rate   | 0.0001    |
|    n_updates       | 2298      |
----------------------------------
Logging to ./logs/sac/reptile_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 352       |
|    ep_rew_mean     | -1.79e+03 |
| time/              |           |
|    episodes        | 200       |
|    fps             | 547       |
|    time_elapsed    | 26        |
|    total_timesteps | 64704     |
| train/             |    

Logging to ./logs/sac/reptile_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 114      |
|    ep_rew_mean     | -395     |
| time/              |          |
|    episodes        | 1500     |
|    fps             | 534      |
|    time_elapsed    | 3        |
|    total_timesteps | 301616   |
| train/             |          |
|    actor_loss      | 213      |
|    critic_loss     | 252      |
|    ent_coef        | 0.169    |
|    ent_coef_loss   | -8.76    |
|    learning_rate   | 0.0001   |
|    n_updates       | 18844    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 107      |
|    ep_rew_mean     | -332     |
| time/              |          |
|    episodes        | 1600     |
|    fps             | 529      |
|    time_elapsed    | 24       |
|    total_timesteps | 312768   |
| train/             |          |
|    actor_loss      | 219      |
|    critic_loss

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 67.6     |
|    ep_rew_mean     | -70.5    |
| time/              |          |
|    episodes        | 3000     |
|    fps             | 526      |
|    time_elapsed    | 58       |
|    total_timesteps | 430688   |
| train/             |          |
|    actor_loss      | 208      |
|    critic_loss     | 321      |
|    ent_coef        | 0.12     |
|    ent_coef_loss   | 0.0833   |
|    learning_rate   | 0.0001   |
|    n_updates       | 26911    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 74       |
|    ep_rew_mean     | -68.4    |
| time/              |          |
|    episodes        | 3100     |
|    fps             | 527      |
|    time_elapsed    | 71       |
|    total_timesteps | 437952   |
| train/             |          |
|    actor_loss      | 206      |
|    critic_loss     | 345      |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 51       |
|    ep_rew_mean     | -146     |
| time/              |          |
|    episodes        | 4500     |
|    fps             | 515      |
|    time_elapsed    | 57       |
|    total_timesteps | 529472   |
| train/             |          |
|    actor_loss      | 173      |
|    critic_loss     | 323      |
|    ent_coef        | 0.145    |
|    ent_coef_loss   | 0.223    |
|    learning_rate   | 0.0001   |
|    n_updates       | 33085    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 55.1     |
|    ep_rew_mean     | -193     |
| time/              |          |
|    episodes        | 4600     |
|    fps             | 515      |
|    time_elapsed    | 67       |
|    total_timesteps | 534736   |
| train/             |          |
|    actor_loss      | 175      |
|    critic_loss     | 311      |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 61.3     |
|    ep_rew_mean     | -65.9    |
| time/              |          |
|    episodes        | 6000     |
|    fps             | 524      |
|    time_elapsed    | 18       |
|    total_timesteps | 609648   |
| train/             |          |
|    actor_loss      | 147      |
|    critic_loss     | 280      |
|    ent_coef        | 0.15     |
|    ent_coef_loss   | -0.259   |
|    learning_rate   | 0.0001   |
|    n_updates       | 38096    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 56.7     |
|    ep_rew_mean     | -69.9    |
| time/              |          |
|    episodes        | 6100     |
|    fps             | 525      |
|    time_elapsed    | 29       |
|    total_timesteps | 615392   |
| train/             |          |
|    actor_loss      | 148      |
|    critic_loss     | 345      |
|    ent_coef 

...

Logging to ./logs/sac/reptile_0
Logging to ./logs/sac/reptile_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 457       |
|    ep_rew_mean     | 1.53e+05  |
| time/              |           |
|    episodes        | 82100     |
|    fps             | 556       |
|    time_elapsed    | 6         |
|    total_timesteps | 22903824  |
| train/             |           |
|    actor_loss      | -3.06e+04 |
|    critic_loss     | 3.16e+06  |
|    ent_coef        | 3.84      |
|    ent_coef_loss   | -0.297    |
|    learning_rate   | 0.0001    |
|    n_updates       | 1431482   |
----------------------------------
Logging to ./logs/sac/reptile_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 524       |
|    ep_rew_mean     | 2.13e+05  |
| time/              |           |
|    episodes        | 82200     |
|    fps             | 557       |
|    time_elapsed    | 5         |
|    total_timesteps | 229528

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 138       |
|    ep_rew_mean     | 2.14e+04  |
| time/              |           |
|    episodes        | 83500     |
|    fps             | 552       |
|    time_elapsed    | 75        |
|    total_timesteps | 23491776  |
| train/             |           |
|    actor_loss      | -3.07e+04 |
|    critic_loss     | 5.43e+04  |
|    ent_coef        | 3.75      |
|    ent_coef_loss   | -0.306    |
|    learning_rate   | 0.0001    |
|    n_updates       | 1468229   |
----------------------------------
Logging to ./logs/sac/reptile_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 271      |
|    ep_rew_mean     | 7.92e+04 |
| time/              |          |
|    episodes        | 83600    |
|    fps             | 553      |
|    time_elapsed    | 23       |
|    total_timesteps | 23512848 |
| train/             |          |
|    actor_loss      | -3.1e+04 |

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 69.1      |
|    ep_rew_mean     | -829      |
| time/              |           |
|    episodes        | 84900     |
|    fps             | 555       |
|    time_elapsed    | 69        |
|    total_timesteps | 23788736  |
| train/             |           |
|    actor_loss      | -3.01e+04 |
|    critic_loss     | 1.52e+05  |
|    ent_coef        | 3.99      |
|    ent_coef_loss   | -0.0997   |
|    learning_rate   | 0.0001    |
|    n_updates       | 1486789   |
----------------------------------
Logging to ./logs/sac/reptile_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 424      |
|    ep_rew_mean     | 1.96e+05 |
| time/              |          |
|    episodes        | 85000    |
|    fps             | 553      |
|    time_elapsed    | 16       |
|    total_timesteps | 23808864 |
| train/             |          |
|    actor_loss      | -3e+04   |

Logging to ./logs/sac/reptile_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 391       |
|    ep_rew_mean     | 1.69e+05  |
| time/              |           |
|    episodes        | 86300     |
|    fps             | 552       |
|    time_elapsed    | 13        |
|    total_timesteps | 24107344  |
| train/             |           |
|    actor_loss      | -3.06e+04 |
|    critic_loss     | 6.48e+04  |
|    ent_coef        | 3.63      |
|    ent_coef_loss   | -0.104    |
|    learning_rate   | 0.0001    |
|    n_updates       | 1506702   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 71.5      |
|    ep_rew_mean     | 1.05e+03  |
| time/              |           |
|    episodes        | 86400     |
|    fps             | 554       |
|    time_elapsed    | 66        |
|    total_timesteps | 24136704  |
| train/             |           |
|    actor_loss      | 

KeyboardInterrupt: 

In [12]:
model.save("trained_models/g1-reptile/sac")